# Linked Data

Testing with Omeka S API

## Setup

In [1]:
import requests
import rdflib
from rdflib import Graph, plugin, URIRef, Literal
from rdflib.serializer import Serializer
from rdflib.plugin import register, Parser
from rdflib import Namespace, RDF, BNode, plugin, Variable, FOAF, SDO

In [2]:
# create sample data to add to the graph
newData = {
    'J.D. Salinger' : { 
        'https://schema.org/deathDate' : 2010,
        'https://schema.org/deathPlace': 'https://en.wikipedia.org/wiki/New_York_City'
        },
    'Herman Melvill' : { 
        'https://schema.org/deathDate' : 1862,
        'https://schema.org/deathPlace' : 'https://en.wikipedia.org/wiki/New_York_City'
        }
}

## Searching in Omeka S

Search for all the items in the specified set

In [3]:
url = 'http://jajohnst.si676.si.umich.edu/omeka-s/api'

action = '/items'

parameters = {
#    property%5B0%5D%5Bjoiner%5D=and
#    property%5B0%5D%5Bproperty%5D=
#    property%5B0%5D%5Btype%5D=eq
#    property%5B0%5D%5Btext%5D=
#    resource_class_id%5B%5D=
#    resource_template_id%5B%5D=
    'item_set_id':440,
#    site_id=
#    owner_id=
#    'fulltext_search':'',
#    'submit':'Search'
}

In [4]:
r = requests.get(url + action, params=parameters)

print(r.url)
print(r.status_code)

http://jajohnst.si676.si.umich.edu/omeka-s/api/items?item_set_id=440
200


In [5]:
r.json()

[{'@context': 'http://jajohnst.si676.si.umich.edu/omeka-s/api-context',
  '@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438',
  '@type': ['o:Item', 'dcterms:Agent'],
  'o:id': 438,
  'o:is_public': True,
  'o:owner': {'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/users/1',
   'o:id': 1},
  'o:resource_class': {'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/resource_classes/1',
   'o:id': 1},
  'o:resource_template': None,
  'o:thumbnail': None,
  'o:title': 'Test record for J.D. Salinger RDF example',
  'thumbnail_display_urls': {'large': None, 'medium': None, 'square': None},
  'o:created': {'@value': '2024-11-18T01:11:42+00:00',
   '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
  'o:modified': {'@value': '2024-11-18T01:15:28+00:00',
   '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
  'o:primary_media': None,
  'o:media': [],
  'o:item_set': [{'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/item_sets/440',
    'o:id': 440}],
  'o:

## Parse data with the RDFLib module

Using the `rdflib` module capabilities, parse this data.

First, create an RDF graph from it:

In [6]:
g = Graph().parse(data=r.text, format='json-ld')

Now, look through the graph. The graph is a series of "triples",
which are subject-predicate-object tuples. These can be modified. example, after the initial look, you can remove all of those with the Omeka S namespace (`o`).
Note that RDFLib may drop or delete any orphaned subjects or objects that may not be part of a triple. 

In [7]:
for s, p, o in g:
    print(f'{s} -> {p} -> {o}.')

http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439 -> http://omeka.org/s/vocabs/o#id -> 439.
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439 -> http://omeka.org/s/vocabs/o#modified -> 2024-11-18T01:15:28+00:00.
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://omeka.org/s/vocabs/o#Item.
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://purl.org/dc/terms/Agent.
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438 -> http://omeka.org/s/vocabs/o#is_public -> true.
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438 -> http://omeka.org/s/vocabs/o#id -> 438.
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438 -> http://omeka.org/s/vocabs/o#created -> 2024-11-18T01:11:42+00:00.
http://jajohnst.si676.si.umich.edu/omeka-s/api/sites/1 -> http://omeka.org/s/vocabs/o#id -> 1.
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439 ->

### Outputting, saving, and serializing

Convert the graph to 'Turtle' format

In [8]:
ser = g.serialize(format='turtle')

print(ser)

@prefix ns1: <http://omeka.org/s/vocabs/o#> .
@prefix ns2: <http://purl.org/dc/terms/> .
@prefix ns3: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438> a ns1:Item,
        ns2:Agent ;
    ns1:created "2024-11-18T01:11:42+00:00"^^xsd:dateTime ;
    ns1:id 438 ;
    ns1:is_public true ;
    ns1:item_set <http://jajohnst.si676.si.umich.edu/omeka-s/api/item_sets/440> ;
    ns1:modified "2024-11-18T01:15:28+00:00"^^xsd:dateTime ;
    ns1:owner <http://jajohnst.si676.si.umich.edu/omeka-s/api/users/1> ;
    ns1:resource_class <http://jajohnst.si676.si.umich.edu/omeka-s/api/resource_classes/1> ;
    ns1:site <http://jajohnst.si676.si.umich.edu/omeka-s/api/sites/1> ;
    ns1:title "Test record for J.D. Salinger RDF example" ;
    ns2:title "Test record for J.D. Salinger RDF example" ;
    ns3:name "J.D. Salinger" .

<http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439> a ns1:Item,
        ns2:Agent 

Save it to a file

In [9]:
with open('item-set-graph-1.ttl', 'w') as f:
    f.write(ser)

## Parsing, Modifying, and Adding to the Graph

Now try to remove the Omeka data in order to get a closer look
at the collection specific data.

In [10]:
# remove the omeka specific data
for triple in g:
    if 'http://omeka.org/s/vocabs/o#' in triple[1]:
        g.remove(triple)

In [11]:
for s, p, o in g:
    print(f'{s} -> {p} -> {o}')

http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://omeka.org/s/vocabs/o#Item
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://omeka.org/s/vocabs/o#Item
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438 -> http://purl.org/dc/terms/title -> Test record for J.D. Salinger RDF example
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://purl.org/dc/terms/Agent
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438 -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://purl.org/dc/terms/Agent
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/438 -> http://xmlns.com/foaf/0.1/name -> J.D. Salinger
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439 -> http://xmlns.com/foaf/0.1/name -> Herman Melvill
http://jajohnst.si676.si.umich.edu/omeka-s/api/items/439 -> http://purl.org

### Adding the "newData"

In [12]:
# add the "newData" by looping (iterating) through the data
# and adding to the appropriate elements

# Note: this will only work if the Keys are in the data already on the site,
# so the data must be uploaded and added first

for nameKey in newData: 
    for s, p, o, in g.triples((None, FOAF.name, Literal(nameKey))):
        deathDate = newData[o.value]['https://schema.org/deathDate']
        deathPlace = newData[o.value]['https://schema.org/deathPlace']
        g.add((s, URIRef('https://schema.org/deathDate'), Literal(deathDate)))
        g.add((s, URIRef('https://schema.org/deathPlace'), URIRef(deathPlace)))

To demonstrate how the graph changed, serialize the new graph

In [13]:
ser2 = g.serialize(format='turtle')

with open('item-set-graph-2.ttl', 'w') as f:
    f.write(ser2)